In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
from pathlib import Path

In [4]:
import sys
sys.path.append("/dsmlp/home-fs04/19/019/riling/scalable_rl_portfolio_management")


In [5]:
from tics.tic_config import tics_176, tics_grouped
from agent.data_downloader import short_name_sha256

In [6]:
tics = tics_176
train_start_date= '2009-01-01'
train_end_date= '2020-07-01'
test_start_date= '2020-07-01'
test_end_date= '2021-10-01'

In [7]:
start = train_start_date
end = test_end_date

In [39]:
df = pd.read_csv(Path("data") / "sub" /  f"{short_name_sha256('_'.join(tics_grouped[1]))}_{start}_{end}.csv").rename(columns={"close":'account_value'})

In [40]:
def normalize_from(df: pd.DataFrame, start_date: str, end_date) -> pd.DataFrame:
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'])
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    
    df = df[df['date'] >= start]
    df = df[df['date'] <= end].sort_values('date')
    
    mask = df['date'] == start
    if not mask.any():
        raise ValueError(f"没有找到日期 {start_date} 的记录")
    V0 = df.loc[mask, 'account_value'].iloc[0]
    
    # 4. 归一化
    df['account_value'] = df['account_value'] / V0
    
    return df

In [41]:
df_normalized = normalize_from(df, '2009-03-17', test_end_date)

In [42]:
import plotly.graph_objects as go
import plotly.express as px

fig = go.Figure()

color_sequence = px.colors.qualitative.Pastel

models = [df_normalized[df_normalized['tic'] == tic].copy() for tic in tics_grouped[1]]

for i, (df, label) in enumerate(zip(models, tics_grouped[1])):
    days = list(range(1, len(df) + 1))
    fig.add_trace(
        go.Scatter(
            # x=df[x_col],
            x=days,
            y=df['account_value'],
            mode='lines',
            name=label,
            line=dict(color=color_sequence[i % len(color_sequence)], width=2),
            opacity=0.6
        )
    )

fig.show()

In [46]:
def parse_weights(s: str):
        return np.array([float(x) for x in s.strip('[]').split()])

df = pd.read_csv(Path("weights") / f"{short_name_sha256('_'.join(tics_grouped[1]))}_{start}_{end}.csv")
df = (
    df
    .assign(w_arr=lambda d: d['weights'].apply(parse_weights))
)

In [51]:
df['w_arr'].apply(sum).min()

0.9999998000000001